In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime
import math
import gc

from sklearn.metrics import roc_auc_score

import lightgbm as lgbm

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data


from IPython.display import clear_output

python(36187) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [3]:
df = load_pkl('./data/feat_engin/lgbm/data_15min_1hour_1day.pkl')

In [5]:
df.shape

(2340439, 1712)

In [14]:
i = 28960
df.iloc[i:i+20]

,time,close,volume,ticker,tmos_close,index_1hour,index_1day,result,delta_time,income_rate,...,tmos_close_1day_w10_min/tmos_close_1day_w10_max,close_1day_w20_min/close_1day_w20_max,volume_1day_w20_min/volume_1day_w20_max,tmos_close_1day_w20_min/tmos_close_1day_w20_max,close_1day_w30_min/close_1day_w30_max,tmos_close_1day_w30_min/tmos_close_1day_w30_max,close_1day_w60_min/close_1day_w60_max,tmos_close_1day_w60_min/tmos_close_1day_w60_max,close_1day_w120_min/close_1day_w120_max,tmos_close_1day_w120_min/tmos_close_1day_w120_max
28960,2024-06-21 16:30:00,63.919998,26437.0,AFLT,6.46,7242.0,539.0,LOSE,2 days 17:45:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28961,2024-06-21 16:45:00,63.799999,7332.0,AFLT,6.46,7243.0,539.0,LOSE,2 days 17:30:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28962,2024-06-21 17:00:00,63.619999,25455.0,AFLT,6.44,7243.0,539.0,LOSE,0 days 06:45:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28963,2024-06-21 17:15:00,63.570000,55831.0,AFLT,6.45,7243.0,539.0,LOSE,0 days 06:30:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28964,2024-06-21 17:30:00,63.500000,25125.0,AFLT,6.45,7243.0,539.0,LOSE,0 days 06:15:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28965,2024-06-21 17:45:00,63.450001,81603.0,AFLT,6.44,7244.0,539.0,LOSE,0 days 05:45:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28966,2024-06-21 18:00:00,63.330002,75504.0,AFLT,6.43,7244.0,539.0,LOSE,0 days 04:45:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28967,2024-06-21 18:15:00,63.470001,24025.0,AFLT,6.43,7244.0,539.0,LOSE,0 days 05:15:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28968,2024-06-21 18:30:00,63.410000,13547.0,AFLT,6.44,7244.0,539.0,LOSE,0 days 05:00:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293
28969,2024-06-21 18:45:00,63.410000,3547.0,AFLT,6.44,7245.0,539.0,LOSE,0 days 04:45:00,0.994,...,0.944277,0.83354,0.173054,0.889362,0.779969,0.874477,0.630124,0.874477,0.54146,0.870293


## CHANGE TARGET

In [9]:
def get_target(df_all, ind, val_first, val_second, points_to_wait):
    #цель - достичь val_second, не достигнув val_first

    ind_end = min(ind+points_to_wait+1, df_all.shape[0])
    
    df = df_all.iloc[ind:ind_end].copy()
    
    mask_stock = np.array(df['ticker'] == df['ticker'].iloc[0])
    df = df.loc[mask_stock, :]

    start_price = df['close'].iloc[0]
    
    if val_first < val_second:                                 
        mask_val_first = np.array(df['close'] < val_first)
        mask_val_second = np.array(df['close'] > val_second)
    else: #short game
        mask_val_first = np.array(df['close'] > val_first)
        mask_val_second = np.array(df['close'] < val_second)

    
    if (mask_val_first.sum() == 0) & (mask_val_second.sum() == 0):

        ind_end = df.shape[0]-1
        delta_time = df['time'].iloc[ind_end] - df['time'].iloc[0]
        res_price = df['close'].iloc[ind_end]
        if val_first < val_second:
            income_rate = res_price/start_price
        else: #short game
            income_rate = (2*start_price-res_price)/start_price
        return 'DNF', delta_time, income_rate, res_price, ind_end+ind
        
    if (mask_val_first.sum() == 0) & (mask_val_second.sum() != 0):
        ind_val_second = np.argwhere(mask_val_second).ravel()[0]
        delta_time = df['time'].iloc[ind_val_second] - df['time'].iloc[0]
        res_price = df['close'].iloc[ind_val_second]
        if val_first < val_second:
            income_rate = res_price/start_price
        else: #short game
            income_rate = (2*start_price-res_price)/start_price
        return 'WIN', delta_time, income_rate, res_price, ind_val_second+ind
        
    if (mask_val_first.sum() != 0) & (mask_val_second.sum() == 0):
        ind_val_first = np.argwhere(mask_val_first).ravel()[0]
        delta_time = df['time'].iloc[ind_val_first] - df['time'].iloc[0]
        res_price = df['close'].iloc[ind_val_first]
        if val_first < val_second:
            income_rate = res_price/start_price
        else: #short game
            income_rate = (2*start_price-res_price)/start_price
        return 'LOSE', delta_time, income_rate, res_price, ind_val_first+ind

    if (mask_val_first.sum() != 0) & (mask_val_second.sum() != 0):
        ind_val_first = np.argwhere(mask_val_first).ravel()[0]
        ind_val_second = np.argwhere(mask_val_second).ravel()[0]
        if ind_val_first < ind_val_second:
            delta_time = df['time'].iloc[ind_val_first] - df['time'].iloc[0]
            res_price = df['close'].iloc[ind_val_first]
            if val_first < val_second:
                income_rate = res_price/start_price
            else: #short game
                income_rate = (2*start_price-res_price)/start_price
            return 'LOSE', delta_time, income_rate, res_price, ind_val_first+ind
        if ind_val_first > ind_val_second:
            delta_time = df['time'].iloc[ind_val_second] - df['time'].iloc[0]
            res_price = df['close'].iloc[ind_val_second]
            if val_first < val_second:
                income_rate = res_price/start_price
            else:  #short game
                income_rate = (2*start_price-res_price)/start_price
            return 'WIN', delta_time, income_rate, res_price, ind_val_second+ind


def get_df_target(df, indx, percent_first=None, percent_second=None, points_to_wait=None):
    times = []
    results = []
    delta_times = []
    income_rates = []
    closes = []
    tickers = []
    res_prices = []
    res_inds = []
    
    #for ind in indx:
    for ind in tqdm(indx):
        time = df['time'].iloc[ind]
        close = df['close'].iloc[ind]
        ticker = df['ticker'].iloc[ind]
        
        val_first = df['close'].iloc[ind] * percent_first
        val_second = df['close'].iloc[ind] * percent_second
        result, delta_time, income_rate, res_price, res_ind = get_target(df, ind, val_first, val_second, points_to_wait)
        
        times += [time]
        closes += [close]
        tickers += [ticker]
        results += [result]
        delta_times += [delta_time]
        income_rates += [income_rate]
        res_prices += [res_price]
        res_inds += [res_ind]
        

    df_result = pd.DataFrame({'ind' : indx,
                              'time' : times,
                              'close' : closes,
                              'result' : results,
                              'ticker' : tickers, 
                              'delta_time' : delta_times,
                              'income_rate' : income_rates,
                              'res_price' : res_prices,
                              'res_ind' : res_inds
                             })

    #макс выигрыш и потери
    if percent_first < percent_second:  
        df_result["income_rate"] = np.maximum(df_result["income_rate"], percent_first) #макс потери
        df_result["income_rate"] = np.minimum(df_result["income_rate"], percent_second) #макс выигрыш
    else:
        df_result["income_rate"] = np.maximum(df_result["income_rate"], 2-percent_first) #макс потери
        df_result["income_rate"] = np.minimum(df_result["income_rate"], 2-percent_second) #макс выигрыш
    
    df_result['income_rate'] -= 0.001
    
    return df_result    

In [12]:
inds = np.arange(df.shape[0])
inds.shape

(2340439,)

In [13]:
#subsample
coeff_subsample = 0.1
inds_subsample = np.random.RandomState(seed=42).permutation(inds)[:int(len(inds)*coeff_subsample)]
inds_subsample.shape


(234043,)

In [ ]:
percent_first = 0.98
percent_second = 1.05
points_to_wait = 4*14*2

df_result = get_df_target(df[['time', 'close', 'ticker']], inds_subsample, 
                          percent_first=percent_first, percent_second=percent_second, points_to_wait=points_to_wait)

In [44]:
df_result['result'].value_counts(normalize=True)

result
DNF     0.526412
LOSE    0.368278
WIN     0.105310
Name: proportion, dtype: float64

In [46]:
df_result['income_rate'].mean()

1.0006708

In [120]:
percents = [(0.995, 1.01),
             (0.995, 1.015),
           (0.99, 1.01),
            (0.99, 1.015),
           (0.99, 1.02),
            (0.99, 1.025),
           (0.99, 1.03),
           (0.98, 1.02),
            (0.98, 1.025),
           (0.98, 1.03),
            (0.98, 1.035),
           (0.98, 1.04),
            (0.98, 1.045),
           (0.98, 1.05),
            
           (1.005, 0.99),
            (1.005, 0.985),
           (1.01, 0.99),
            (1.01, 0.985),
           (1.01, 0.98),
            (1.01, 0.975),
           (1.01, 0.97),
           (1.02, 0.98),
            (1.02, 0.975),
           (1.02, 0.97),
            (1.02, 0.965),
           (1.02, 0.96),
            (1.02, 0.955),
           (1.02, 0.95),
           ]

points = [4*2*1,
          4*4*1,
          4*6*1,
          4*8*1,
          4*14*1,
          4*14*2,
          4*14*3,
          4*14*5,
         ]




df_research = {'percents' : [],
              'points_to_wait' : [],
              'income_rate_mean' : [],
              'income_rate_win' : [],
              'strategy' : [],
              }


pbar = tqdm(total=len(percents)*len(points))
for percent_first, percent_second in percents:
    for points_to_wait in points:
        df_result = get_df_target(df[['time', 'close', 'ticker']], inds_subsample, 
                              percent_first=percent_first, percent_second=percent_second, points_to_wait=points_to_wait)


        df_research['percents'] += [(percent_first, percent_second)]
        df_research['points_to_wait'] += [points_to_wait]
        df_research['income_rate_mean'] += [df_result['income_rate'].mean()]
        df_research['income_rate_win'] += [
            (abs(percent_second-1)-0.001)*df_result['result'].value_counts(normalize=True)['WIN'] - (abs(percent_first-1) + 0.001)*(1-df_result['result'].value_counts(normalize=True)['WIN'])]
        if percent_first < percent_second:
            df_research['strategy'] += ['long']
        else:
            df_research['strategy'] += ['short']


        #clear_output()
        pbar.update(1)
pbar.close()


dump_pkl(df_research, 'df_research.pkl')


100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4208.30it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4216.52it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4265.64it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4256.15it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4194.94it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4223.01it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4333.96it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:53<00:00, 4388.62it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4220.

In [122]:
df_research = pd.DataFrame(df_research)

In [124]:
df_research

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
0,"(0.995, 1.01)",8,0.999299,-0.004341,long
1,"(0.995, 1.01)",16,0.999371,-0.003226,long
2,"(0.995, 1.01)",24,0.999408,-0.002481,long
3,"(0.995, 1.01)",32,0.999428,-0.001972,long
4,"(0.995, 1.01)",56,0.999452,-0.001196,long
...,...,...,...,...,...
219,"(1.02, 0.95)",32,0.999510,-0.019690,short
220,"(1.02, 0.95)",56,0.999909,-0.018160,short
221,"(1.02, 0.95)",112,1.000194,-0.014393,short
222,"(1.02, 0.95)",168,1.000311,-0.011428,short


#### income_rate_mean

In [135]:
df_research.sort_values(by='income_rate_mean', ascending=False).iloc[:20]

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
103,"(0.98, 1.45)",280,1.005450,-0.019891,long
102,"(0.98, 1.45)",168,1.003207,-0.020412,long
101,"(0.98, 1.45)",112,1.002072,-0.020683,long
111,"(0.98, 1.05)",280,1.001908,-0.006325,long
95,"(0.98, 1.04)",280,1.001210,-0.004804,long
110,"(0.98, 1.05)",168,1.001145,-0.010625,long
55,"(0.99, 1.03)",280,1.000973,-0.000322,long
87,"(0.98, 1.035)",280,1.000795,-0.004101,long
100,"(0.98, 1.45)",56,1.000763,-0.020867,long
54,"(0.99, 1.03)",168,1.000750,-0.001906,long


#### income_rate_win

In [152]:
df_research.sort_values(by='income_rate_win', ascending=False)

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
127,"(1.005, 0.985)",280,1.000090,0.000009,short
15,"(0.995, 1.015)",280,0.999966,-0.000095,long
126,"(1.005, 0.985)",168,1.000092,-0.000166,short
14,"(0.995, 1.015)",168,0.999955,-0.000251,long
47,"(0.99, 1.025)",280,1.000544,-0.000271,long
...,...,...,...,...,...
100,"(0.98, 1.45)",56,1.000763,-0.020867,long
99,"(0.98, 1.45)",32,0.999955,-0.020944,long
98,"(0.98, 1.45)",24,0.999718,-0.020960,long
97,"(0.98, 1.45)",16,0.999462,-0.020976,long


In [159]:
df_research.sort_values(by='income_rate_win', ascending=False).iloc[:40]

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
127,"(1.005, 0.985)",280,1.000090,0.000009,short
15,"(0.995, 1.015)",280,0.999966,-0.000095,long
126,"(1.005, 0.985)",168,1.000092,-0.000166,short
14,"(0.995, 1.015)",168,0.999955,-0.000251,long
47,"(0.99, 1.025)",280,1.000544,-0.000271,long
55,"(0.99, 1.03)",280,1.000973,-0.000322,long
119,"(1.005, 0.99)",280,0.999643,-0.000379,short
39,"(0.99, 1.02)",280,1.000075,-0.000389,long
118,"(1.005, 0.99)",168,0.999641,-0.000437,short
125,"(1.005, 0.985)",112,1.000097,-0.000446,short


In [163]:
# 125 	(1.005, 0.985) 	112 	1.000097 	-0.000446 	short
# 13 	(0.995, 1.015) 	112 	0.999920 	-0.000548 	long

# v2

In [22]:
percents = [
             (0.995, 1.015),
           (0.995, 1.02),
            (0.995, 1.025),

            
           (1.005, 0.985),
            (1.005, 0.98),
           (1.005, 0.975),
  
           ]

points = [
          4*8*1,
          4*14*1,
          4*14*2,
          4*14*3,
          4*14*5,
         ]




df_research = {'percents' : [],
              'points_to_wait' : [],
              'income_rate_mean' : [],
              'income_rate_win' : [],
              'strategy' : [],
              }


pbar = tqdm(total=len(percents)*len(points))
for percent_first, percent_second in percents:
    for points_to_wait in points:
        df_result = get_df_target(df[['time', 'close', 'ticker']], inds_subsample, 
                              percent_first=percent_first, percent_second=percent_second, points_to_wait=points_to_wait)


        df_research['percents'] += [(percent_first, percent_second)]
        df_research['points_to_wait'] += [points_to_wait]
        df_research['income_rate_mean'] += [df_result['income_rate'].mean()]
        df_research['income_rate_win'] += [
            (abs(percent_second-1)-0.001)*df_result['result'].value_counts(normalize=True)['WIN'] - (abs(percent_first-1) + 0.001)*(1-df_result['result'].value_counts(normalize=True)['WIN'])]
        if percent_first < percent_second:
            df_research['strategy'] += ['long']
        else:
            df_research['strategy'] += ['short']


        #clear_output()
        pbar.update(1)
pbar.close()


dump_pkl(df_research, 'df_research.pkl')


100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:53<00:00, 4351.96it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4285.48it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4275.72it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:52<00:00, 4419.00it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:53<00:00, 4389.71it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4255.42it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4235.21it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4252.68it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:53<00:00, 4373.

In [24]:
df_research = pd.DataFrame(df_research)

### income_rate_win

In [27]:
df_research.sort_values(by='income_rate_win', ascending=False)

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
14,"(0.995, 1.025)",280,1.000749,0.000468,long
29,"(1.005, 0.975)",280,1.000700,0.000335,short
9,"(0.995, 1.02)",280,1.000393,0.000251,long
24,"(1.005, 0.98)",280,1.000440,0.000248,short
19,"(1.005, 0.985)",280,1.000090,0.000009,short
8,"(0.995, 1.02)",168,1.000357,-0.000081,long
13,"(0.995, 1.025)",168,1.000679,-0.000083,long
23,"(1.005, 0.98)",168,1.000432,-0.000084,short
4,"(0.995, 1.015)",280,0.999966,-0.000095,long
18,"(1.005, 0.985)",168,1.000092,-0.000166,short


# v3

In [24]:
percents = [
             (0.995, 1.015),
           # (0.995, 1.02),
           #  (0.995, 1.025),

            
           (1.005, 0.985),
           #  (1.005, 0.98),
           # (1.005, 0.975),
  
           ]

points = [
          4*2*1,
          4*4*1, 
          4*8*1,
          4*14*1,
          4*14*2,

         ]




df_research = {'percents' : [],
              'points_to_wait' : [],
              'income_rate_mean' : [],
              'income_rate_win' : [],
              'strategy' : [],
              }


pbar = tqdm(total=len(percents)*len(points))
for percent_first, percent_second in percents:
    for points_to_wait in points:
        df_result = get_df_target(df[['time', 'close', 'ticker']], inds_subsample, 
                              percent_first=percent_first, percent_second=percent_second, points_to_wait=points_to_wait)


        df_research['percents'] += [(percent_first, percent_second)]
        df_research['points_to_wait'] += [points_to_wait]
        df_research['income_rate_mean'] += [df_result['income_rate'].mean()]
        df_research['income_rate_win'] += [
            (abs(percent_second-1)-0.001)*df_result['result'].value_counts(normalize=True)['WIN'] - (abs(percent_first-1) + 0.001)*(1-df_result['result'].value_counts(normalize=True)['WIN'])]
        if percent_first < percent_second:
            df_research['strategy'] += ['long']
        else:
            df_research['strategy'] += ['short']


        #clear_output()
        pbar.update(1)
pbar.close()


dump_pkl(df_research, 'df_research.pkl')


100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4182.82it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:56<00:00, 4161.01it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4302.59it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4252.29it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4237.22it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4214.49it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4227.86it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:54<00:00, 4271.28it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4238.

In [25]:
df_research = pd.DataFrame(df_research)

In [30]:
#income_rate win
df_research.sort_values(by='income_rate_win', ascending=False)

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
9,"(1.005, 0.985)",112,1.000097,-0.000446,short
4,"(0.995, 1.015)",112,0.999920,-0.000548,long
8,"(1.005, 0.985)",56,1.000078,-0.001385,short
3,"(0.995, 1.015)",56,0.999852,-0.001443,long
7,"(1.005, 0.985)",32,0.999947,-0.002524,short
2,"(0.995, 1.015)",32,0.999771,-0.002528,long
1,"(0.995, 1.015)",16,0.999626,-0.003881,long
6,"(1.005, 0.985)",16,0.999774,-0.003914,short
0,"(0.995, 1.015)",8,0.999480,-0.004865,long
5,"(1.005, 0.985)",8,0.999583,-0.004926,short


# v4

In [22]:
percents = [
    (0.995, 1.015),
    (0.995, 1.02),
    (0.99, 1.02),
    (0.99, 1.03),
    (0.998, 1.04),
    (0.998, 1.05),
           ]

points = [
          4*4*1, 
          4*8*1,
          4*14*1,
          4*14*2,

         ]




df_research = {'percents' : [],
              'points_to_wait' : [],
              'income_rate_mean' : [],
              'income_rate_win' : [],
              'strategy' : [],
              }


pbar = tqdm(total=len(percents)*len(points))
for percent_first, percent_second in percents:
    for points_to_wait in points:
        df_result = get_df_target(df[['time', 'close', 'ticker']], inds_subsample, 
                              percent_first=percent_first, percent_second=percent_second, points_to_wait=points_to_wait)


        df_research['percents'] += [(percent_first, percent_second)]
        df_research['points_to_wait'] += [points_to_wait]
        df_research['income_rate_mean'] += [df_result['income_rate'].mean()]
        df_research['income_rate_win'] += [
            (abs(percent_second-1)-0.001)*df_result['result'].value_counts(normalize=True)['WIN'] - (abs(percent_first-1) + 0.001)*(1-df_result['result'].value_counts(normalize=True)['WIN'])]
        if percent_first < percent_second:
            df_research['strategy'] += ['long']
        else:
            df_research['strategy'] += ['short']


        #clear_output()
        pbar.update(1)
pbar.close()


dump_pkl(df_research, 'df_research.pkl')


100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:56<00:00, 4122.78it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4190.71it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4183.45it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:56<00:00, 4169.77it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4208.09it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4205.22it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:56<00:00, 4177.37it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:56<00:00, 4176.61it/s]

100%|███████████████████████████████████████████████████████████████| 234043/234043 [00:55<00:00, 4222.

In [23]:
df_research = pd.DataFrame(df_research)

In [24]:
#income_rate win
df_research.sort_values(by='income_rate_win', ascending=False)

,percents,points_to_wait,income_rate_mean,income_rate_win,strategy
19,"(0.998, 1.04)",112,1.000969,-0.000393,long
3,"(0.995, 1.015)",112,0.999920,-0.000548,long
7,"(0.995, 1.02)",112,1.000298,-0.000573,long
23,"(0.998, 1.05)",112,1.001119,-0.000770,long
18,"(0.998, 1.04)",56,1.000846,-0.001319,long
2,"(0.995, 1.015)",56,0.999852,-0.001443,long
22,"(0.998, 1.05)",56,1.000960,-0.001683,long
6,"(0.995, 1.02)",56,1.000176,-0.001842,long
17,"(0.998, 1.04)",32,1.000603,-0.001997,long
21,"(0.998, 1.05)",32,1.000673,-0.002258,long
